<b>Description:</b> Generate an optimized portfolio of equities at a given risk level.

In [1]:
## Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import quandl
import datetime

# API key: 8b9yt7Fa2LUNy3s8QTjN
quandl.ApiConfig.api_key = '8b9yt7Fa2LUNy3s8QTjN'
